In [65]:
# %pip install -qU langchain-community pypdf

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import fitz  # PyMuPDF
import os
import hashlib
import base64
from openai import OpenAI

In [11]:
client = OpenAI()

In [7]:
def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")
        
        
def extract_text_and_images(pdf_path, output_folder, min_width=100, min_height=100, min_filesize=5000):
    """
    Extract text and unique images from each page of a PDF into a dictionary.
    Skips small icons/logos and duplicate images (by size + dimensions).
    """
    doc = fitz.open(pdf_path)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    seen = set()  # store (width, height, filesize) triples
    results = {}  # dictionary to hold page data
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        page_dict = {"text": "", "images": []}
        
        # --- Extract text ---
        text = page.get_text("text").strip()
        page_dict["text"] = text if text else ""
        
        # --- Extract images ---
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            width, height = img[2], img[3]
            
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            filesize = len(image_bytes)
            
            # Skip icons/logos (small dimension or tiny file size)
            if width < min_width or height < min_height or filesize < min_filesize:
                continue
            
            # Skip duplicates based on (dimensions + file size)
            key = (width, height, filesize)
            if key in seen:
                continue
            seen.add(key)
            
            # Save unique image
            output_path = os.path.join(
                output_folder, f"page_{page_num+1}_img_{img_index+1}.{image_ext}"
            )
            with open(output_path, "wb") as f:
                f.write(image_bytes)
            
            page_dict["images"].append(output_path)
        
        results[page_num + 1] = page_dict  # store by page number
    
    doc.close()
    return results

In [9]:
pdf_data = extract_text_and_images("/home/jovyan/projects/data/helpkasra.pdf", "extracted_images")

# Print summary
for page, content in pdf_data.items():
    print(f"\n--- Page {page} ---")
    print("Text:", content["text"][:200], "...")  # preview first 200 chars
    print("Images:", content["images"])

    break


--- Page 1 ---
Text: 1 
 
 
 
 
 
 
 
 راهنمای
عمومی سامانه حضور و غیاب کسرا 
 
)(تردد ...
Images: ['extracted_images/page_1_img_1.jpeg', 'extracted_images/page_1_img_2.jpeg']


In [12]:
# Example input
item = {
    "text": """4 

کسرای من  

صفحه کسرای من با  نماد مشاهده میشود .
: شرح عملکرد 
نوار ابزار  عمودی 
شامل
آیکن صفحاتی 
است که به آن دسترسی دارید که با کلیک بر روی هر آ یکن نام صفحه نمایش
داده میشود برای مشاهده نام کلیه صفحات کافیست بر روی عالمت . کلیک نمایید 
کلیه
پیامهای 
ارسالی 
مدیر سیستم  و صفحات پر کاربرد انتخاب شده در
میزکار، 
در
این صفحه 
مطابق تصویر 
قابل مشاهده
. است 
میزکار توسط شما قابل تنظیم.میباشد 
توضیحات تکمیلی در خصوص صفحه
میزکار را از  اینجا  
مشاهده نمایید.""",
    "images": ["extracted_images/page_4_img_3.png"]
}

content = [
    {"type": "text", "text": f"لطفاً این متن و تصاویر همراه آن را بررسی کرده و یک توضیح کامل و بدون مقدمه ارائه بده:\n\n{item['text']}"}
]

# اضافه کردن تصاویر
for img in item["images"]:
    b64 = encode_image(img)
    content.append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{b64}"
        }
    })

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "تو یک دستیار هوشمند هستی که متن و تصویر را بررسی و توضیح می‌دهی."},
        {"role": "user", "content": content}
    ]
)

print(response.choices[0].message.content)

متن و تصویر ارائه‌شده به معرفی صفحه "کسرای من" در یک سیستم نرم‌افزاری می‌پردازد. در این صفحه، کاربران با استفاده از نوار ابزار عمودی می‌توانند به صفحات مختلف دسترسی پیدا کنند. آیکن‌های موجود در نوار ابزار به کاربران این امکان را می‌دهند که با کلیک بر روی هر کدام، نام صفحه مورد نظر را مشاهده کنند.

**عناصر کلیدی صفحه:**

1. **میزکار:** کاربران می‌توانند میزکار خود را طبق سلیقه‌شان تنظیم کنند. این بخش شامل پیام‌های ارسالی مدیر سیستم و همچنین صفحات پرکاربرد انتخابی است.

2. **پیام‌ها:** قسمت ویژه‌ای برای مشاهده پیام‌های مدیر سیستم وجود دارد که می‌توان با کلیک بر روی یک آیکن، پیام‌های قبلی و بعدی را مشاهده کرد.

3. **وضعیت مجوزها:** در بخشی دیگر، اطلاعاتی درباره وضعیت مجوزها و تعداد مجوزهای تایید شده و رد شده ارائه می‌شود.

4. **گراف اطلاعات:** نمودار دایره‌ای در گوشه صفحه اطلاعاتی درباره تعداد درخواست‌ها و وضعیت‌های مختلف را به تصویر می‌کشد.

این صفحه به کاربران کمک می‌کند تا به راحتی به اطلاعات و امکانات مختلف دسترسی داشته باشند و وضعیت مجوزهای خود را پیگیری کنند.


In [14]:
pdf_data[4]

{'text': '4 \n \n \n کسرای من  \n \n  صفحه کسرای من با  نماد            مشاهده میشود . \n : شرح عملکرد \n نوار ابزار  عمودی \n شامل\nآیکن صفحاتی \n است که به آن دسترسی دارید که با کلیک بر روی هر آ یکن نام صفحه نمایش\n داده میشود برای مشاهده نام کلیه صفحات کافیست بر روی عالمت \n \n . کلیک نمایید \n کلیه\nپیامهای \n ارسالی \n مدیر سیستم  و صفحات پر کاربرد انتخاب شده در\nمیزکار، \n در\nاین صفحه \n مطابق تصویر \n قابل مشاهده\n . است \n میزکار توسط شما قابل تنظیم.میباشد \n توضیحات تکمیلی در خصوص صفحه\nمیزکار را از  اینجا  \n مشاهده نمایید.',
 'images': ['extracted_images/page_4_img_3.png']}

In [15]:
item = pdf_data[4]

content = [
    {"type": "text", "text": f"لطفاً این متن و تصاویر همراه آن را بررسی کن و **توضیح کامل بده**، بدون هیچ مقدمه‌ای:\n\n{item['text']}"}
]

for img in item["images"]:
    b64 = encode_image(img)
    content.append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{b64}"
        }
    })

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "تو یک دستیار هوشمند هستی که متن و تصویر را بررسی و توضیح می‌دهی، بدون هیچ جملهٔ اضافی یا مقدمه."},
        {"role": "user", "content": content}
    ]
)

print(response.choices[0].message.content)

متن و تصویر ارائه‌شده مربوط به رابط کاربری یک سیستم مدیریتی است. توضیحات در مورد صفحه‌ی "کسرای من" و امکانات آن به شرح زیر است:

1. **صفحه‌ی اصلی (کسرای من)**:
   - صفحه با یک نماد خاص به نمایش درآمده و شامل نوار ابزار عمودی است که امکان دسترسی به صفحات مختلف را فراهم می‌کند.
   - با کلیک بر روی هر آیکون، نام صفحه مربوطه نمایش داده می‌شود و برای مشاهده نام تمامی صفحات، کاربر می‌تواند بر روی علامت مشخص شده کلیک کند.

2. **پیام‌ها**:
   - در این صفحه، پیام‌های ارسالی از سوی مدیر سیستم و همچنین صفحات پرکاربرد مشخص‌شده در میزکار قابل مشاهده است.

3. **تنظیمات میزکار**:
   - میزکار قابل تنظیم توسط کاربر بوده و امکان شخصی‌سازی را فراهم می‌آورد.

4. **جزئیات رابط کاربری**:
   - در بالای صفحه، نوار جستجو و بخش‌های مختلف مانند «درخواست‌های متمرکز» و «عملکرد‌های متمرکز» قراردارد.
   - پیوندها و آیکون‌هایی برای دسترسی به سایر خدمات (پیام‌ها، تنظیمات و ...) در سمت راست وجود دارد.

5. **بخش آمار و اطلاعات**:
   - اطلاعات مربوط به وضعیت مجوزها در زیر بخش "کارت‌های من" نشان داده شده است، از جمله تعدا

In [18]:
item = pdf_data[7]

content = [
    {"type": "text", "text": f"لطفاً این متن و تصاویر همراه آن را بررسی کن و **توضیح کامل بده**، بدون هیچ مقدمه‌ای:\n\n{item['text']}"}
]

for img in item["images"]:
    b64 = encode_image(img)
    content.append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{b64}"
        }
    })

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "تو یک دستیار هوشمند هستی که متن و تصویر را بررسی و توضیح می‌دهی، بدون هیچ جملهٔ اضافی یا مقدمه."},
        {"role": "user", "content": content}
    ]
)

print(response.choices[0].message.content)

متن ارائه شده راهنمایی برای جستجوی صفحات در یک سیستم نرم‌افزاری است. در اولین بخش توضیح داده شده که چطور می‌توان با کلیک بر روی آیکن جستجو در پنل سمت راست، کادر جستجو را فعال کرد. سپس کاربران می‌توانند بخشی از نام صفحه مورد نظر را تایپ کنند تا لیست صفحاتی که شامل آن نام هستند و دسترسی دارند، مشاهده کنند.

در ادامه بیان شده که زیر نام صفحه مورد نظر، مسیر دستیابی از طریق منو نمایش داده می‌شود. همچنین، استفاده از کلید ترکیبی SHIFT+F برای دسترسی سریع به کادر جستجو در کلیه صفحات نرم‌افزار اشاره شده است.

تصویر ارائه شده به نمایش رابط کاربری نرم‌افزار می‌پردازد. آیکن جستجو در سمت راست مشخص شده و قسمت‌های مختلف پنل کاربری نیز قابل مشاهده است، از جمله لیست درخواست‌ها و نمودارهایی که به تجزیه و تحلیل داده‌ها کمک می‌کند.
